In [15]:
!pip install pymongo

In [6]:
from pymongo import MongoClient

# Inisialisasi client dan koneksi ke database
client = MongoClient('mongodb://localhost:27017/')
db = client['company_db']
collection = db['employees']

# Contoh untuk memastikan koneksi
print('Connected to MongoDB')

Connected to MongoDB


In [7]:
#Menghapus Semua Data Lama
collection.delete_many({})
print("Koleksi dikosongkan sebelum menambahkan data baru.\n")

#Proses INSERT
print("Proses INSERT")
employees_to_add = [
    {'name': 'Bob', 'department': 'IT', 'age': 35, 'salary': 7000, 'status': 'Permanent'},
    {'name': 'Charlie', 'department': 'Marketing', 'age': 23, 'salary': 5000, 'status': 'Permanent'},
    {'name': 'Musa', 'department': 'IT', 'age': 42, 'salary': 8500, 'status': 'Permanent'},
    {'name': 'Karezu', 'department': 'Marketing', 'age': 25, 'salary': 5000, 'status': 'Permanent'},
    {'name': 'Rita', 'department': 'Finance', 'age': 29, 'salary': 4800, 'status': 'Permanent'},
    {'name': 'Evelyn', 'department': 'Finance', 'age': 33, 'salary': 6200, 'status': 'Permanent'}
]

result = collection.insert_many(employees_to_add)
print(f"Berhasil menambahkan {len(result.inserted_ids)} data karyawan baru.")

#Proses UPDATE
print("\nProses UPDATE")
query_filter = { 'name': 'Bob' }
new_data = { "$set": { "salary": 7500 } }
collection.update_one(query_filter, new_data)
print("Berhasil meng-update gaji Bob.")
print("\nData Setelah Gaji Bob Di-update")
for employee in collection.find():
    print(employee)

#Proses DELETE
print("\nProses DELETE")
delete_filter = { 'name': 'Charlie' }
collection.delete_one(delete_filter)
print("Berhasil menghapus data Charlie.")

print("\nData Akhir di Koleksi")
for employee in collection.find():
    print(employee)


Koleksi dikosongkan sebelum menambahkan data baru.

Proses INSERT
Berhasil menambahkan 6 data karyawan baru.

Proses UPDATE
Berhasil meng-update gaji Bob.

Data Setelah Gaji Bob Di-update
{'_id': ObjectId('690b58949860e31c83cb0438'), 'name': 'Bob', 'department': 'IT', 'age': 35, 'salary': 7500, 'status': 'Permanent'}
{'_id': ObjectId('690b58949860e31c83cb0439'), 'name': 'Charlie', 'department': 'Marketing', 'age': 23, 'salary': 5000, 'status': 'Permanent'}
{'_id': ObjectId('690b58949860e31c83cb043a'), 'name': 'Musa', 'department': 'IT', 'age': 42, 'salary': 8500, 'status': 'Permanent'}
{'_id': ObjectId('690b58949860e31c83cb043b'), 'name': 'Karezu', 'department': 'Marketing', 'age': 25, 'salary': 5000, 'status': 'Permanent'}
{'_id': ObjectId('690b58949860e31c83cb043c'), 'name': 'Rita', 'department': 'Finance', 'age': 29, 'salary': 4800, 'status': 'Permanent'}
{'_id': ObjectId('690b58949860e31c83cb043d'), 'name': 'Evelyn', 'department': 'Finance', 'age': 33, 'salary': 6200, 'status': 'Pe

In [8]:
pipeline = [
    {'$group': {'_id': '$department', 'average_salary': { '$avg': '$salary' }}}
]

for result in collection.aggregate(pipeline):
    print(result)

{'_id': 'IT', 'average_salary': 8000.0}
{'_id': 'Marketing', 'average_salary': 5000.0}
{'_id': 'Finance', 'average_salary': 5500.0}


In [9]:
#Tugas 5
employees_to_add = [
    {'name': 'Charlie', 'department': 'Marketing', 'age': 23, 'salary': 5000, 'status': 'Permanent'},
]

result = collection.insert_many(employees_to_add)

In [10]:
print("5 Karyawan Gaji Tertinggi per Departemen")

top_5_salary_pipeline = [
    # Tahap 1: Urutkan karyawan berdasarkan gaji tertinggi dulu dalam tiap departemen
    {'$sort': {'department': 1, 'salary': -1}},
    
    # Tahap 2: Kelompokkan berdasarkan departemen
    {'$group': {'_id': '$department','employees': {'$push': {'name': '$name','salary': '$salary'}}}},
    
    # Tahap 3: Ambil hanya 5 karyawan teratas untuk setiap departemen
    {'$project': {'top_5_employees': {'$slice': ['$employees', 5]}}}
]

# Jalankan agregasi dan cetak hasilnya
for result in collection.aggregate(top_5_salary_pipeline):
    print(f"\nDepartemen: {result['_id']}")
    
    for employee in result['top_5_employees']:
        print(f"  - Nama: {employee['name']}, Gaji: {employee['salary']}")



5 Karyawan Gaji Tertinggi per Departemen

Departemen: Finance
  - Nama: Evelyn, Gaji: 6200
  - Nama: Rita, Gaji: 4800

Departemen: IT
  - Nama: Musa, Gaji: 8500
  - Nama: Bob, Gaji: 7500

Departemen: Marketing
  - Nama: Karezu, Gaji: 5000
  - Nama: Charlie, Gaji: 5000


In [11]:
print("Karyawan yang akan dihapus (usia < 25 tahun)")
delete_filter = {'age': {'$lt': 25}}  # Hapus usia di bawah 25

# Ambil data yang sesuai
employees_to_delete = collection.find(delete_filter)

# Hitung jumlah data
count_to_delete = collection.count_documents(delete_filter)

if count_to_delete > 0:
    for emp in employees_to_delete:
        print(f"- Nama: {emp['name']}, Usia: {emp['age']}")
    
    # Eksekusi penghapusan
    print(f"\nMenghapus {count_to_delete} karyawan...")
    result = collection.delete_many(delete_filter)
    
    # Tampilkan jumlah yang dihapus
    print(f"Berhasil dihapus: {result.deleted_count} data karyawan.")
else:
    print("Tidak ada karyawan dengan usia di bawah 25 tahun.")

Karyawan yang akan dihapus (usia < 25 tahun)
- Nama: Charlie, Usia: 23

Menghapus 1 karyawan...
Berhasil dihapus: 1 data karyawan.


In [14]:
print("Laporan Ringkas Gaji dan Usia per Departemen")

summary_report_pipeline = [
    # Tahap 1: Kelompokkan dan hitung total gaji serta rata-rata usia
    {'$group': {'_id': '$department','total_salary': { '$sum': '$salary' },'average_age': { '$avg': '$age' }}},
    # Tahap 2 (Opsional): Urutkan laporan berdasarkan nama departemen
    {'$sort': { '_id': 1 }}
]

# Jalankan agregasi dan cetak hasilnya
for report in collection.aggregate(summary_report_pipeline):
    department = report['_id']
    total_salary = report['total_salary']
    avg_age = report['average_age']
    
    print(f"\nDepartemen: {department}")
    print(f"  - Total Gaji: ${total_salary:,.2f}")
    print(f"  - Rata-rata Usia: {avg_age:.1f} tahun")


Laporan Ringkas Gaji dan Usia per Departemen

Departemen: Finance
  - Total Gaji: $11,000.00
  - Rata-rata Usia: 31.0 tahun

Departemen: IT
  - Total Gaji: $16,000.00
  - Rata-rata Usia: 38.5 tahun

Departemen: Marketing
  - Total Gaji: $5,000.00
  - Rata-rata Usia: 25.0 tahun
